# Light GBM baseline

Heavily taken from:  https://www.kaggle.com/sz8416/lb-1-4439-gacr-prediction-eda-lgb-baseline and from Olivier 

### Results and notes
- Best LB score: 1.4378 using visitStartTime and local hour of day, highly tuned hyper parameters. CV score 1.6415 session and 1.6126 user level

- Current last model: LB = 1.4404, CV = 1.6167, 1.5919, and visit start time + local hour of day + many categorical interaction features
- Top 10 categorical interactions, removed visitStartTime, added year, day of week, time since last session:
        - Session level CV score:  1.6242508497927641
        - User level CV score:  1.6031847639160595


In [1]:
# load basic packages
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp
import warnings
import matplotlib.pyplot as plt
plt.style.use("seaborn")
import seaborn as sns
import spacy
nlp = spacy.load("en")

# machine learning packages
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import lightgbm as lgb

# save model
import pickle

# set environment
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_Colwidth", -1)
warnings.filterwarnings("ignore")

/Users/nityansuman/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
data = pd.read_csv("data/train.csv")

In [4]:
data.columns

Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')

In [5]:
data.head(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western Asia"", ""country"": ""Turkey"", ""region"": ""Izmir"", ""metro"": ""(not set)"", ""city"": ""Izmir"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""ttnet.com.tr"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Macintosh"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Australasia"", ""country"": ""Australia"", ""region"": ""not available in demo dataset"", ""metro"": ""not available in demo dataset"", ""city"": ""not available in demo dataset"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""dodo.net.au"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": "

In [ ]:
# Extract target values and Ids
cat_cols = ['channelGrouping','device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent','trafficSource.adContent',
       #'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       #'trafficSource.adwordsClickInfo.isVideoAd',
       #'trafficSource.adwordsClickInfo.page',
       #'trafficSource.adwordsClickInfo.slot', #Drop as only 3 values and always poor
       'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source',
        #Interaction columns
        'geoNetwork.city+geoNetwork.networkDomain',
        'device.operatingSystem+geoNetwork.networkDomain',
        'device.operatingSystem+geoNetwork.city', 
        'channelGrouping+geoNetwork.networkDomain',
        'geoNetwork.city+trafficSource.source',
        'geoNetwork.networkDomain+trafficSource.source',
        'geoNetwork.networkDomain+trafficSource.referralPath',
        'geoNetwork.networkDomain+trafficSource.medium',
        'geoNetwork.city+trafficSource.medium',
        'geoNetwork.city+geoNetwork.country',
        #Time columns (categorical)
         '_dayofweek', '_year'
           ]

to_drop = ['trafficSource.adwordsClickInfo.adNetworkType','trafficSource.adwordsClickInfo.isVideoAd',
          'trafficSource.adwordsClickInfo.page','trafficSource.adwordsClickInfo.slot']

num_cols = ['visitNumber', 'totals.bounces', 'totals.hits',
            'totals.newVisits', 'totals.pageviews', '_time_since_last_visit' ]

#interaction_cols = ['totals.hits / totals.pageviews']

visitStartTime = ['visitStartTime']

time_cols = [ '_local_hourofday' , ]

ID_cols = ['date', 'fullVisitorId', 'sessionId', 'visitId']

target_col = ['totals.transactionRevenue']



In [ ]:
train_df = pd.read_pickle('../input/kernel-for-file-processing-2/train_flat_FE_CAT_LE.pkl')
test_df = pd.read_pickle('../input/kernel-for-file-processing-2/test_flat_FE_CAT_LE.pkl')

#train_df['_time_since_last_visit'] = pd.to_numeric(train_df['_time_since_last_visit'])
#test_df['_time_since_last_visit'] = pd.to_numeric(test_df['_time_since_last_visit'])

train_df.drop(to_drop, axis = 1, inplace = True)
test_df.drop(to_drop, axis = 1, inplace = True)

In [ ]:
#Time features
train_df['_dayofweek'] = train_df['visitStartTime'].dt.dayofweek
train_df['_year'] = train_df['visitStartTime'].dt.year

test_df['_dayofweek'] = test_df['visitStartTime'].dt.dayofweek
test_df['_year'] = test_df['visitStartTime'].dt.year

## Categorical interactions 
- Remove categorical values with less than 10 values (default from LGBM anyways)
- Generate order 2 interactions and keep the top 10
- Rename to "other" category with less than 10 values

In [ ]:
%%time
#Categorical encoding
for c in cat_cols:
    #Convert NAs to unknown
    train_df[c] = train_df[c].fillna('unknown')
    test_df[c] = test_df[c].fillna('unknown')


#Rename "Other" those with less than 10
for col in cat_cols:
    #For train data
    series1 = pd.value_counts(train_df[col])
    mask1 = series1 < 10
    train_df[col] = np.where(train_df[col].isin(series1[mask1].index),'Other_{}'.format(col), train_df[col])
    
    #For test data
    series2 = pd.value_counts(test_df[col])
    mask2 = series2 < 10
    test_df[col] = np.where(test_df[col].isin(series2[mask2].index),'Other_{}'.format(col), test_df[col])

## Other time based features 

In [ ]:
%%time
#train_df['_time_since_last_visit'] = pd.to_numeric(train_df['_time_since_last_visit'])
#test_df['_time_since_last_visit'] = pd.to_numeric(test_df['_time_since_last_visit'])

#Preparation
wip_cols = ['fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime','totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', '_time_since_last_visit']

train_ts = train_df.sort_values(['fullVisitorId', 'visitStartTime']).reset_index()
train_ts['index'] = train_ts['index'].astype('str')
train_ts_grouped = train_ts.groupby('fullVisitorId')

#Calculating rolling frequency
temp_roll = train_ts_grouped.rolling('1H', on ='visitStartTime')['visitNumber'].count().reset_index().add_suffix('_1H') 
train_ts = pd.concat([train_ts, temp_roll['visitNumber_1H']], axis = 1)

temp_roll = train_ts_grouped.rolling('1D', on ='visitStartTime')['visitNumber'].count().reset_index().add_suffix('_1D') 
train_ts = pd.concat([train_ts, temp_roll['visitNumber_1D']], axis = 1)

temp_roll = train_ts_grouped.rolling('30D', on ='visitStartTime')['visitNumber'].count().reset_index().add_suffix('_30D') 
train_ts = pd.concat([train_ts, temp_roll['visitNumber_30D']], axis = 1)



In [ ]:
%%time

test_ts = test_df.sort_values(['fullVisitorId', 'visitStartTime']).reset_index()
test_ts['index'] = test_ts['index'].astype('str')
test_ts_grouped = test_ts.groupby('fullVisitorId')

#Calculating rolling frequency
temp_roll = test_ts_grouped.rolling('1H', on ='visitStartTime')['visitNumber'].count().reset_index().add_suffix('_1H') 
test_ts = pd.concat([test_ts, temp_roll['visitNumber_1H']], axis = 1)

temp_roll = test_ts_grouped.rolling('1D', on ='visitStartTime')['visitNumber'].count().reset_index().add_suffix('_1D') 
test_ts = pd.concat([test_ts, temp_roll['visitNumber_1D']], axis = 1)

test_roll = test_ts_grouped.rolling('30D', on ='visitStartTime')['visitNumber'].count().reset_index().add_suffix('_30D') 
test_ts = pd.concat([test_ts, temp_roll['visitNumber_30D']], axis = 1)



In [ ]:
%%time

#Factorize cats
for f in (cat_cols):
    train_df[f], indexer = pd.factorize(train_df[f])
    test_df[f] = indexer.get_indexer(test_df[f])

del indexer

In [ ]:
train_df.head()

In [ ]:
%%time
#Numeric as float
for n in [cat_cols + num_cols + time_cols]:
    train_df[n] = train_df[n].fillna(0).astype('int')
    test_df[n] = test_df[n].fillna(0).astype('int')

#train totals.transactionRevenue
train_df['totals.transactionRevenue'] = train_df['totals.transactionRevenue'].fillna(0).astype('float')

#visitStartTime
for v in time_cols:
    train_df[v] = pd.to_numeric(train_df[v])
    test_df[v] = pd.to_numeric(test_df[v])


In [ ]:
#Index
train_idx = train_df['fullVisitorId']
test_idx = test_df['fullVisitorId']

#Targets
train_target = np.log1p(train_df.groupby("fullVisitorId")["totals.transactionRevenue"].sum())
train_y = np.log1p(train_df["totals.transactionRevenue"])

#Datasets
train_X = train_df[cat_cols + num_cols + time_cols]
test_X = test_df[cat_cols + num_cols +  time_cols]


print(train_X.shape)
print(test_X.shape)

In [ ]:
predictions_train = pd.DataFrame(data = {'fullVisitorId':train_df['fullVisitorId'], 
                                         'sessionId':train_df['sessionId'], 
                                         'visitId':train_df['visitId'],
                                         'index':train_df.index,
                                         'totals.transactionRevenue':np.log1p(train_df['totals.transactionRevenue']),
                                         'predictedRevenue':np.nan})

predictions_test = pd.DataFrame(data = {'fullVisitorId':test_df['fullVisitorId'], 
                                         'sessionId':test_df['sessionId'], 
                                         'visitId':test_df['visitId'],
                                         'index':test_df.index, 
                                         'predictedRevenue':np.nan})


In [ ]:
from lightgbm import LGBMRegressor

#Initialize LGBM
gbm = LGBMRegressor(objective = 'regression', 
                     boosting_type = 'gbdt', 
                     metric = 'rmse',
                     n_estimators = 10000, #10000
                     num_leaves = 30,
                     learning_rate = 0.01, #0.01
                     bagging_fraction = 0.9,#0.8
                     feature_fraction = 0.3,#.3
                     #bagging_seed = 42,
                     #max_depth = 12, #-1 
                     #categorical_feature = [train_df[cat_cols].columns.get_loc(c) for c in cat_cols ] ,
                     #cat_smooth = 20
                    )

In [ ]:
%%time
#Initilization
all_K_fold_results = []
kf = KFold(n_splits=5, shuffle = True)
k = 0


for i, v in kf.split(train_X):
    dev_index, val_index  = train_X.index[i], train_X.index[v]
    X_dev, X_val = train_X.loc[dev_index], train_X.loc[val_index]
    y_dev, y_val = train_y[dev_index], train_y[val_index]
    
    #Fit the model
    model = gbm.fit(X_dev,y_dev, eval_set=[(X_val, y_val)],verbose = 100, 
                    eval_metric = 'rmse', early_stopping_rounds = 100) #100
    
    #Predict out of fold 
    predictions_train.loc[val_index, 'predictedRevenue'] = gbm.predict(X_val, num_iteration= model.best_iteration_)
    predictions_train[predictions_train['predictedRevenue'] < 0] = 0
        
    #Predict on test set based on current fold model. Average results
    sub_prediction = pd.Series(data = gbm.predict(test_X, num_iteration= model.best_iteration_))
    sub_prediction[sub_prediction<0] = 0
    predictions_test['Predictions_{}'.format(k)] = sub_prediction.copy()
    k += 1 #increase by 1
    
    #Save current fold values
    fold_results = {'best_iteration_' : model.best_iteration_, 
                   'best_score_' : model.best_score_['valid_0']['rmse'], 
                   'evals_result_': model.evals_result_['valid_0']['rmse'],
                   'feature_importances_' : model.feature_importances_}

    all_K_fold_results.append(fold_results.copy())
    

#Save results
results = pd.DataFrame(all_K_fold_results)
predictions_test['average_predictions'] = predictions_test.iloc[:,-5:].mean(axis = 1).copy()



In [ ]:
def RMSE_log_sum(pred, df):
    #set negative values to zero
    pred[pred < 0] = 0
    
    #Build new dataframe
    pred_df = pd.DataFrame(data = {'fullVisitorId': df['fullVisitorId'].values, 
                                       'transactionRevenue': df['totals.transactionRevenue'].values,
                                      'predictedRevenue':np.expm1(pred) })
    #Compute sum
    pred_df = pred_df.groupby('fullVisitorId').sum().apply(np.log1p).reset_index()

    mse_log_sum = mean_squared_error( pred_df['transactionRevenue'].values, 
                             pred_df['predictedRevenue'].values)  

    
    #print('log (sum + 1): ',np.sqrt(mse_log_sum))
    return np.sqrt(mse_log_sum)


def save_submission(pred_test, test_df, file_name):
    #Zero negative predictions
    pred_test[pred_test < 0] = 0
    
    #Create temporary dataframe
    sub_df = pd.DataFrame(data = {'fullVisitorId':test_df['fullVisitorId'], 
                             'predictedRevenue':np.expm1(pred_test)})
    sub_df = sub_df.groupby('fullVisitorId').sum().apply(np.log1p).reset_index()
    sub_df.columns = ['fullVisitorId', 'predictedLogRevenue']
    #sub_df['predictedLogRevenue'] = np.log1p(sub_df['predictedLogRevenue'])
    sub_df.to_csv(file_name, index = False)

    
def visualize_results(results):
#Utility function to plot fold loss and best model feature importance
    plt.figure(figsize=(16, 12))

    #----------------------------------------
    # Plot validation loss
    plt.subplot(2,2,1)

    for K in range(results.shape[0]):
        plt.plot(np.arange(len(results.evals_result_[K])), results.evals_result_[K], label = 'fold {}'.format(K))

    plt.xlabel('Boosting iterations')
    plt.ylabel('RMSE')
    plt.title('Validation loss vs boosting iterations')
    plt.legend()

    #----------------------------------------
    # Plot box plot of RMSE
    plt.subplot(2, 2, 2)    
    scores = results.best_score_
    plt.boxplot(scores)
    rmse_mean = np.mean(scores)
    rmse_std = np.std(scores)
    plt.title('RMSE Mean:{:.3f} Std: {:.4f}'.format(rmse_mean,rmse_std ))
    
    #----------------------------------------
    # Plot feature importance
    #feature_importance = results.sort_values('best_score_').feature_importances_[0]
    df_feature_importance = pd.DataFrame.from_records(results.feature_importances_)
    feature_importance = df_feature_importance.mean()
    std_feature_importance = df_feature_importance.std()
    
    # make importances relative to max importance
    #feature_importance = 100.0 * (mean_feature_importance / mean_feature_importance.sum())
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.subplot(2, 1, 2)
    plt.bar(pos, feature_importance[sorted_idx], align='center', yerr = std_feature_importance)
    xlabels = [ test_X.columns.values[i] for i in sorted_idx]
    plt.xticks(pos, xlabels, rotation = 90)
    plt.xlabel('Feature')
    plt.ylabel('Avg Importance score')
    plt.title('Mean Feature Importance over K folds') 
    
    plt.show()

### Previous iteration
Session level CV score:  1.6112586662249373
User level CV score:  1.5919866631295672

In [ ]:
print('Session level CV score (filtered data): ', np.mean(results.best_score_))
print('Session level CV score (all data):', np.sqrt(mean_squared_error(predictions_train['totals.transactionRevenue'], predictions_train['predictedRevenue'])))
print('User level CV score: ', RMSE_log_sum(predictions_train['predictedRevenue'],train_df))

In [ ]:
visualize_results(results)

In [ ]:
train_df['visitStartTime'] = pd.to_datetime(train_df['visitStartTime'])

error_df = pd.DataFrame(data = {'visitStartTime':train_df['visitStartTime'],'fullVisitorId':train_df['sessionId'], 
                                'True_log_revenue' : np.log1p(train_df['totals.transactionRevenue']), 
                                'Predicted_log_revenue':predictions_train['predictedRevenue']  })

error_df['Difference'] = error_df['True_log_revenue'] - error_df['Predicted_log_revenue']
error_df['True_is_non_zero'] = error_df['True_log_revenue'] > 0
#temp_df.columns = ['fullVisitorId', 'predictedLogRevenue']
#sub_df['predictedLogRevenue'] = np.log1p(sub_df['predictedLogRevenue'])
#sub_df.to_csv(file_name, index = False)
error_df.head(5).sort_values('True_log_revenue')

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,7))


sns.distplot(error_df[error_df['True_is_non_zero'] == False]['True_log_revenue'], ax = ax1, label = 'true')
sns.distplot(error_df[error_df['True_is_non_zero'] == False ]['Predicted_log_revenue'], ax = ax1, label = 'pred')
ax1.legend()
ax1.set_ylim(0,.1)
ax1.set_xlabel('Log revenue (session)')
ax1.set_title('Distribution of log revenues for sessions with zero true revenue ')

sns.distplot(error_df[error_df['True_is_non_zero'] == True]['True_log_revenue'], ax = ax2, label = 'true')
sns.distplot(error_df[error_df['True_is_non_zero'] == True ]['Predicted_log_revenue'], ax = ax2, label = 'pred')
ax2.legend()
ax2.set_ylim(0,.5)
ax2.set_xlabel('Log revenue (session)')
ax2.set_title('Distribution of log revenues for sessions with non zero true revenue ')

plt.show()

In [ ]:
sorted_non_zero = error_df[error_df['True_is_non_zero'] == True].sort_values('visitStartTime')


plt.figure(figsize = (20,15))
plt.subplot(2,2,1)
plt.plot(sorted_non_zero.visitStartTime, sorted_non_zero.True_log_revenue , label = 'True')
plt.plot(sorted_non_zero.visitStartTime, sorted_non_zero.Predicted_log_revenue , alpha = .5, label = 'Pred')
plt.title('Log revenue over time (non zero true sessions only)')
plt.legend()
plt.xlabel('Time: sessions')

plt.subplot(2,2,2)
daily_error_non_zero_df = sorted_non_zero.set_index('visitStartTime', drop = True).resample('D').mean()
plt.plot(daily_error_non_zero_df.index, daily_error_non_zero_df.True_log_revenue , label = 'True')
plt.plot(daily_error_non_zero_df.index, daily_error_non_zero_df.Predicted_log_revenue , label = 'Pred')
plt.title('Daily average log revenue (non zero true sessions only)')

plt.subplot(2,2,3)
weekly_error_df = error_df.set_index('visitStartTime', drop = True).resample('W').mean()
plt.plot(weekly_error_df.index, weekly_error_df.True_log_revenue , label = 'True')
plt.plot(weekly_error_df.index, weekly_error_df.Predicted_log_revenue , label = 'Pred')
plt.title('Weekly average log revenue (all session)')


plt.subplot(2,2,4)
daily_error_df = error_df.set_index('visitStartTime', drop = True).resample('D').mean()
plt.plot(daily_error_df.index, daily_error_df.True_log_revenue , label = 'True')
plt.plot(daily_error_df.index, daily_error_df.Predicted_log_revenue , label = 'Pred')
plt.title('Daily average log revenue (all session)')

plt.legend()
plt.show()

In [ ]:
sorted_non_zero = error_df[error_df['True_is_non_zero'] == True].sort_values('visitStartTime')
sorted_zero = error_df[error_df['True_is_non_zero'] == False].sort_values('visitStartTime')


plt.figure(figsize = (20,5))
plt.subplot(1,3,1)
ts_error_df = error_df.set_index('visitStartTime', drop = True)
difference_rev_df = error_df.sort_values('visitStartTime')
plt.plot(error_df.visitStartTime, error_df.Difference , label = 'True - predicted', color = 'grey')
plt.title('Train - Pred (log rev) for all sessions')

plt.subplot(1,3,2)
plt.plot(sorted_non_zero.visitStartTime, sorted_non_zero.Difference , label = 'True - predicted',
         color = 'grey')
plt.title('Train - Pred for non zero sessions only')

plt.subplot(1,3,3)
plt.plot(sorted_zero.visitStartTime, sorted_zero.Difference,
         color = 'grey')
plt.title('Train - Pred for zero sessions only')

plt.legend()
plt.show()

In [ ]:
sns.jointplot(x="True_log_revenue", y="Predicted_log_revenue", data=sorted_non_zero)
display('Joint distribution of log rev for non zero sessions only')

plt.show()

In [ ]:
save_submission(predictions_test['average_predictions'], test_df, 'submission.csv')

In [ ]:
test_df.head()

In [ ]:

#test_df['PredictedLogRevenue'] = predictions_test['average_predictions']
tt = test_df.groupby('fullVisitorId').sum() #.apply(np.log1p).reset_index()

In [ ]:
tt.shape

In [ ]:
test_df.shape

In [ ]:
train_df.shape

In [ ]:
ss = pd.read_csv('../input/ga-customer-revenue-prediction/test.csv')
ss.shape

In [ ]:
    pred_test[pred_test < 0] = 0
    
    #Create temporary dataframe
    sub_df = pd.DataFrame(data = {'fullVisitorId':test_df['fullVisitorId'], 
                             'predictedRevenue':np.expm1(pred_test)})
    sub_df = sub_df.groupby('fullVisitorId').sum().apply(np.log1p).reset_index()
    sub_df.columns = ['fullVisitorId', 'predictedLogRevenue']
    #sub_df['predictedLogRevenue'] = np.log1p(sub_df['predictedLogRevenue'])
    sub_df.to_csv(file_name, index = False)
    
    test['PredictedLogRevenue'] = sub_reg_preds
test[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum()['PredictedLogRevenue'].apply(np.log1p).reset_index()\
    .to_csv('test_clf_reg_log_of_sum.csv', index=False)

## Best features 

In [ ]:
feature_importance = results.feature_importances_.mean()
sorted_idx = np.argsort(feature_importance)
feature_importance[sorted_idx]
feature_names = [ test_X.columns.values[i] for i in sorted_idx]

In [ ]:
importance_df = pd.DataFrame({'features_names': feature_names, 'importance':feature_importance[sorted_idx] })
importance_df.sort_values('importance', ascending = False)['features_names'][0:20].values

In [ ]:
predictions_test['average_predictions'].sum()

### Post processing attempt 

In [ ]:

test_df['average_predictions'] = predictions_test['average_predictions']
predictions_test['totals.bounces'] = test_df['totals.bounces']
error_df['totals.bounces'] = train_df['totals.bounces']


In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,7))


sns.distplot(error_df['True_log_revenue'], ax = ax1, label = 'true')
sns.distplot(error_df['Predicted_log_revenue'], ax = ax1, label = 'pred')
sns.distplot(test_df['average_predictions'], ax = ax1, label = 'test', color = 'yellow')
ax1.legend()
ax1.set_ylim(0,.1)
ax1.set_xlabel('Log revenue (session)')
ax1.set_title('Distribution of log revenues and predictions ')

sns.distplot(error_df[error_df['totals.bounces'] ==1 ]['True_log_revenue'], ax = ax2, label = 'true')
sns.distplot(error_df[error_df['totals.bounces'] ==1 ]['Predicted_log_revenue'], ax = ax2, label = 'pred')
sns.distplot(test_df[test_df['totals.bounces'] ==1]['average_predictions'], ax = ax2, label = 'test', color = 'yellow')
ax2.legend()
ax2.set_ylim(0,.5)
ax2.set_xlabel('Log revenue (session)')
ax2.set_title('Distribution of log revenues for sessions with bounce = 1')

plt.show()

In [ ]:
#Sum of predicted revenues for sessions with a bounce
print('Revenue: True train sessions with bounce =1 : ' ,error_df[error_df['totals.bounces'] ==1 ]['True_log_revenue'].sum(),error_df[error_df['totals.bounces'] ==1 ]['True_log_revenue'].count() ,'sessions')
print('Revenue: Predicted train sessions with bounce =1 : ' , error_df[error_df['totals.bounces'] ==1 ]['Predicted_log_revenue'].sum(), error_df[error_df['totals.bounces'] ==1 ]['Predicted_log_revenue'].count() ,'sessions')
print('Revenue: Predicted test sessions with bounce =1 : ' , test_df[test_df['totals.bounces'] ==1]['average_predictions'].sum(), test_df[test_df['totals.bounces'] ==1 ]['average_predictions'].count() ,'sessions')


In [ ]:
print('Revenue: True train sessions with bounce =0 : ' ,error_df[error_df['totals.bounces'] ==0 ]['True_log_revenue'].sum(), error_df[error_df['totals.bounces'] ==0 ]['True_log_revenue'].count() ,'sessions')
print('Revenue: Predicted train sessions with bounce =0 : ' , error_df[error_df['totals.bounces'] ==0 ]['Predicted_log_revenue'].sum(), error_df[error_df['totals.bounces'] ==0 ]['True_log_revenue'].count() ,'sessions')
print('Revenue: Predicted test sessions with bounce =0 : ' , test_df[test_df['totals.bounces'] ==0]['average_predictions'].sum(), test_df[test_df['totals.bounces'] ==1 ]['average_predictions'].count() ,'sessions')


### Assigning values to zero as post processing 

In [ ]:
predictions_test[predictions_test['totals.bounces'] ==1]['average_predictions'] = 0

In [ ]:
save_submission(predictions_test['average_predictions'], test_df, 'submission_post_process.csv')